In [24]:
import cvxpy as cp
import pandas as pd

costsheet = pd.read_csv('costs.csv') 
costsheet['class1'] = pd.to_numeric(costsheet['class1'], errors='coerce') #csv having some entries in these columns as strings, fix before submission
costsheet['class2'] = pd.to_numeric(costsheet['class2'], errors='coerce')
costsheet['class3'] = pd.to_numeric(costsheet['class3'], errors='coerce')
costsheet['friend1'] = pd.to_numeric(costsheet['friend1'], errors='coerce')
costsheet['friend2'] = pd.to_numeric(costsheet['friend2'], errors='coerce')
costsheet['friend3'] = pd.to_numeric(costsheet['friend3'], errors='coerce')

## HANGOUTS PER MONTH * OPPORTUNITY COST * PROBABILITY ; FOR EACH FRIEND
def friendcost(id):
    return 6*11*(0.025*(costsheet['friend1'][id]/.1)) + 6*18*(0.025*(costsheet['friend2'][id]/.1)) + 6*48*(0.025*(costsheet['friend3'][id]/.1))

## NUMBER OF CLASSES * TUITION PER CLASS * PROBABILITY OF MISSING CLASS, DEPENDENT ON BUYING A LIMEPRIME SUBSCRIPTION
def class_cost(dist):
    if dist > 0.7:
        return 12*51.4*.05*(dist/.75) + 6 + (24*.3*dist)
    else:
        return 12*51.4*.05*(dist/.25) 

## which first class option is closest; for lime scooter purposes 
def closest_class(id):
    return min(costsheet['class1'][id], costsheet['class2'][id], costsheet['class3'][id])
    
## BASE COST OF EACH RENTAL
def basecost(id):
    cost = costsheet['Rent'][id] #rent
    cost = cost + 25 * costsheet['Water'][id] + 50 * costsheet['Electric'][id] + 30 * costsheet['Gas'][id]
    cost = cost + 20 * costsheet['Internet'][id] + 24 * costsheet['Laundry'][id] # utilities
    cost = cost + 13.68 * costsheet['Emergencies'][id] + 6.84 * costsheet['Generals'][id] #landlords
    cost = cost + 13.68 * costsheet['Safety'][id] + 4.16 * costsheet['Sidewalks'][id] + 3.9 * costsheet['Clean'][id]
    cost = cost + costsheet['Off-Street'][id] * costsheet['Fee'][id] #parking

    return float(cost)

## Initialize 36 house choices
x = cp.Variable(36, boolean=True)

constraints = []

constraints.append(sum(x) == 1)

## determine scooter use for each property


## OBJECTIVE FUNCTION: MINIMIZE SUM((RENT + UTILITIES + INTERNET + LAUNDRY + EMERGENCY_MAINTENANCE + GENERAL_MAINTENANCE + SAFETY_COST + ROAD_HAZARD_COST + DIRTY_COST + GARAGE_PARKING + STREET_PARKING + FRIEND1_COST + FRIEND2_COST + FRIEND3_COST) * APARTMENT CHOICE X_i,j 
# + (MORNING_CLASSES_PER_MONTH * (1 - LIME_SCOOTER_DECISION Y)) * SUM(CLASS_0_COST*CLASS_0_CHOICE X_0,j + CLASS_1_COST * CLASS_1_CHOICE X_1,j + CLASS_2_COST * CLASS_2_CHOICE X_2,j) + (MORNING_CLASSES_PER_MONTH * (LIME_SCOOTER_DECISION Y)) * SUM(CLASS_0_COST_LIME*CLASS_0_CHOICE X_0,j + CLASS_1_COST_LIME * CLASS_1_CHOICE X_1,j + CLASS_2_COST_LIME * CLASS_2_CHOICE X_2,j) FOR ALL X_i,j) FOR ALL X_i,j
obj_func =  (
    x[0] * (basecost(1) + friendcost(1) + class_cost(0.8)) + x[1] * (basecost(1) + friendcost(1) + class_cost(1.2)) + x[2] * (basecost(1) + friendcost(1) + class_cost(0.9)) + 
    x[3] * (basecost(2) + friendcost(2) + class_cost(0.3)) + x[4] * (basecost(2) + friendcost(2) + class_cost(0.7)) + x[5] * (basecost(2) + friendcost(2) + class_cost(0.4)) + 
    x[6] * (basecost(3) + friendcost(3) + class_cost(0.7)) + x[7] * (basecost(3) + friendcost(3) + class_cost(0.8)) + x[8] * (basecost(3) + friendcost(3) + class_cost(0.5)) + 
    x[9] * (basecost(4) + friendcost(4) + class_cost(0.9)) + x[10] * (basecost(4) + friendcost(4) + class_cost(1.8)) + x[11] * (basecost(4) + friendcost(4) + class_cost(0.8)) + 
    x[12] * (basecost(5) + friendcost(5) + class_cost(1.5)) + x[13] * (basecost(5) + friendcost(5) + class_cost(1.6)) + x[14] * (basecost(5) + friendcost(5) + class_cost(1.8)) + 
    x[15] * (basecost(6) + friendcost(6) + class_cost(0.6)) + x[16] * (basecost(6) + friendcost(6) + class_cost(0.7)) + x[17] * (basecost(6) + friendcost(6) + class_cost(0.9)) + 
    x[18] * (basecost(7) + friendcost(7) + class_cost(0.8)) + x[19] * (basecost(7) + friendcost(7) + class_cost(1.2)) + x[20] * (basecost(7) + friendcost(7) + class_cost(1.0)) + 
    x[21] * (basecost(8) + friendcost(8) + class_cost(0.8)) + x[22] * (basecost(8) + friendcost(8) + class_cost(1.2)) + x[23] * (basecost(8) + friendcost(8) + class_cost(1.1)) + 
    x[24] * (basecost(9) + friendcost(9) + class_cost(1.0)) + x[25] * (basecost(9) + friendcost(9) + class_cost(0.8)) + x[26] * (basecost(9) + friendcost(9) + class_cost(1.2)) + 
    x[27] * (basecost(10) + friendcost(10) + class_cost(0.7)) + x[28] * (basecost(10) + friendcost(10) + class_cost(0.6)) + x[29] * (basecost(10) + friendcost(10) + class_cost(1.0)) + 
    x[30] * (basecost(11) + friendcost(11) + class_cost(0.9)) + x[31] * (basecost(11) + friendcost(11) + class_cost(0.9)) + x[32] * (basecost(11) + friendcost(11) + class_cost(1.2)) + 
    x[33] * (basecost(12) + friendcost(12) + class_cost(1.7)) + x[34] * (basecost(12) + friendcost(12) + class_cost(1.8)) + x[35] * (basecost(12) + friendcost(12) + class_cost(2.0))
)

problem = cp.Problem(cp.Minimize(obj_func), constraints)

#problem.solve(solver=cp.CVXOPT,verbose = True) #verbose parameter determines showing/not showing the output
problem.solve(solver=cp.GUROBI,verbose = True)

def printHouseClassChoice(value):
    index = value.index(1)
    classChoice = index % 3 + 1
    appChoice = index % 12 + 1
    return [f'''Class {classChoice}, Apartment {appChoice}''']
    
        
        

print(printHouseClassChoice(list(x.value)))
print(x.value, obj_func.value)



    



                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 25 03:17:48 PM: Your problem has 36 variables, 1 constraints, and 0 parameters.
(CVXPY) Nov 25 03:17:48 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 25 03:17:48 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 25 03:17:48 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 25 03:17:48 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 25 03:17:48 PM: Compiling problem (target solver=GUROBI).
(C